#### Import Library

In [56]:
#!/usr/bin/env python       
# coding: utf-8

# importing required libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from os import path
import sklearn
from sklearn import metrics, preprocessing
from sklearn.metrics import (precision_score, recall_score, f1_score, roc_auc_score, 
                             roc_curve, auc, confusion_matrix, accuracy_score, 
                             balanced_accuracy_score, matthews_corrcoef, classification_report)
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder, LabelEncoder, 
                                   MinMaxScaler, OneHotEncoder, Normalizer, 
                                   MaxAbsScaler, RobustScaler, PowerTransformer, LabelBinarizer)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris, make_classification
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import mutual_info_classif
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from lightgbm import LGBMClassifier
import catboost
import xgboost as xgb
from scipy.stats import mode
from tabulate import tabulate
import shap
import joblib

start_program = time.time()

#### Parameters to use or load models, but yet to be implemented, it is not really functional.

In [57]:
# First ensemble with NSL-KDD
# Parameters

#----------------------------------------------
# 0 for not using it as base learner
# 1 for using it as base learner

use_model_ada = 1 
use_model_dnn = 1 
use_model_mlp = 1 
use_model_lgbm = 1 
use_model_rf = 1 
use_model_svm = 1
use_model_knn = 1 
#----------------------------------------------
# 0 for training the model
# 1 for using the saved version of the model

load_model_ada = 0 
load_model_dnn = 0 
load_model_mlp = 0 
load_model_lgbm = 0 
load_model_rf = 0 
load_model_svm = 0
load_model_knn = 0 
#----------------------------------------------


#### Pick name of the file according to parameter

In [58]:
#----------------------------------------------
# feature_selection_bit = 1
feature_selection_bit = 0

# Specify the name of the output text file
if feature_selection_bit == 0:
    output_file_name = "ensemble_base_models_all_features.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('---- ensemble_base_models_all_features', file = f)

elif feature_selection_bit == 1:
    output_file_name = "ensemble_base_models_feature_selection.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('----ensemble_base_models_feature_selection--', file = f)


#### Function definition

In [59]:


def confusion_metrics (name_model,predictions,true_labels,time_taken):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)
    FPR = FP / (FP + TN)
    FPR = 100*(sum(FPR)/len(FPR))

    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)
    with open(output_file_name, "a") as f: print('Time Taken: ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('FPR: ', FPR, '%' ,file = f)

    return Acc, Precision, Recall, F1, BACC, MCC, FPR


#### Load NSLKDD 

In [60]:

# attach the column names to the dataset
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.

train='KDDTrain+.txt'
test='KDDTest+.txt'

df=pd.read_csv(train,names=feature)
df_test=pd.read_csv(test,names=feature)


In [61]:

# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)


df.drop(['difficulty'],axis=1,inplace=True)
df_test.drop(['difficulty'],axis=1,inplace=True)

print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())

# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()


# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()

trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape

newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)


# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())


# Specify your selected features. Note that you'll need to modify this list according to your final processed dataframe
#Uncomment the below lines to use these top 20 features from shap analysis
#selected_features = ["root_shell","service_telnet","num_shells","service_uucp","dst_host_same_src_port_rate"
#                     ,"dst_host_rerror_rate","dst_host_srv_serror_rate","dst_host_srv_count","service_private","logged_in",
#                    "dst_host_serror_rate","serror_rate","srv_serror_rate","flag_S0","diff_srv_rate","dst_host_srv_diff_host_rate","num_file_creations","flag_RSTR"#,"dst_host_same_srv_rate","service_Idap","label"]
                     
    
# Select those features from your dataframe
#newdf = newdf[selected_features]
#newdf_test = newdf_test[selected_features]

# Now your dataframe only contains your selected features.

# creating a dataframe with multi-class labels (Dos,Probe,R2L,U2R,normal)
multi_data = newdf.copy()
multi_label = pd.DataFrame(multi_data.label)

multi_data_test=newdf_test.copy()
multi_label_test = pd.DataFrame(multi_data_test.label)


# using standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = multi_data.select_dtypes(include='number').columns
data = standardization(multi_data,numeric_col)
numeric_col_test = multi_data_test.select_dtypes(include='number').columns
data_test = standardization(multi_data_test,numeric_col_test)

# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
le2_test = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
enc_label_test = multi_label_test.apply(le2_test.fit_transform)
multi_data = multi_data.copy()
multi_data_test = multi_data_test.copy()

multi_data['intrusion'] = enc_label
multi_data_test['intrusion'] = enc_label_test

#y_mul = multi_data['intrusion']
multi_data
multi_data_test

multi_data.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data
multi_data_test.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data_test

y_train_multi= multi_data[['intrusion']]
X_train_multi= multi_data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:',X_train_multi.shape,'\ny_train has shape:',y_train_multi.shape)

y_test_multi= multi_data_test[['intrusion']]
X_test_multi= multi_data_test.drop(labels=['intrusion'], axis=1)

print('X_test has shape:',X_test_multi.shape,'\ny_test has shape:',y_test_multi.shape)

label_counts = Counter(y_train_multi['intrusion'])
print(label_counts)

y_train_multi = LabelBinarizer().fit_transform(y_train_multi)

y_test_multi = LabelBinarizer().fit_transform(y_test_multi)

Y_train=y_train_multi.copy()
X_train=X_train_multi.copy()

Y_test=y_test_multi.copy()
X_test=X_test_multi.copy()

ros = RandomOverSampler(sampling_strategy='minority', random_state=100)

X_train, Y_train = ros.fit_resample(X_train, Y_train)

Dimensions of the Training set: (125973, 43)
Dimensions of the Test set: (22544, 43)
Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

Label distribution Test set:
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess  

In [62]:
single_class_train = np.argmax(y_train_multi, axis=1)
single_class_test = np.argmax(y_test_multi, axis=1)

df1 = X_train_multi.assign(Label = single_class_train)
df2 =  X_test_multi.assign(Label = single_class_test)

frames = [df1,  df2]

df = pd.concat(frames,ignore_index=True)

df = df.sample(15000)


df_fs = df

In [63]:
y = df.pop('Label')
X = df
df = X.assign(Label = y)

## Feature Selection Methods

#### Set feature selection = 1 to use Information gain

In [64]:
if feature_selection_bit == 1:

    # Compute information gain using mutual information
    importances = mutual_info_classif(X, y)

    feat_importances = pd.Series(importances, df.columns[0:len(df.columns)-1])
    # feat_importances.plot(kind='barh', color = 'teal')
        
    feat_importances_sorted = feat_importances.sort_values( ascending=False)

    # Print or use the sorted DataFrame
    print(feat_importances_sorted)
    # feat_importances_sorted.plot(kind='barh', color = 'teal')
    # feat_importances_sorted
    top_features = feat_importances_sorted.nlargest(10)
    top_feature_names = top_features.index.tolist()

    print("Top 10 feature names:")
    print(top_feature_names)


In [65]:

if feature_selection_bit == 1:
    # USE XAI feature selection from last work https://www.mdpi.com/2076-3417/14/10/4170
    feature_selection = [
                        'dst_host_same_srv_rate',
                        'dst_host_srv_count',
                        'dst_host_same_src_port_rate',
                        'logged_in',
                        'dst_host_serror_rate',
                        'count',
                        'srv_count',
                        'dst_host_rerror_rate',
                        'Label'
                        ]
    
    # to Use information gain uncomment line below
    # feature_selection = top_feature_names
    
    df_og = df
    df = df[feature_selection]



##### Create One vs ALL labels that will be needed later

In [66]:

# y = df.pop('Label')
# X = df

y1, y2 = pd.factorize(y)

y_0 = pd.DataFrame(y1)
y_1 = pd.DataFrame(y1)
y_2 = pd.DataFrame(y1)
y_3 = pd.DataFrame(y1)
y_4 = pd.DataFrame(y1)


# y_0 = y_0.replace(0, 0)
# y_0 = y_0.replace(1, 1)
y_0 = y_0.replace(2, 1)
y_0 = y_0.replace(3, 1)
y_0 = y_0.replace(4, 1)


y_1 = y_1.replace(1, 999)
y_1 = y_1.replace(0, 1)
# y_1 = y_1.replace(1, 0)
y_1 = y_1.replace(2, 1)
y_1 = y_1.replace(3, 1)
y_1 = y_1.replace(4, 1)
y_1 = y_1.replace(999, 1)


y_2 = y_2.replace(0, 1)
y_2 = y_2.replace(1, 1)
y_2 = y_2.replace(2, 0)
y_2 = y_2.replace(3, 1)
y_2 = y_2.replace(4, 1)


y_3 = y_3.replace(0, 1)
# y_3 = y_3.replace(1, 1)
y_3 = y_3.replace(2, 1)
y_3 = y_3.replace(3, 0)
y_3 = y_3.replace(4, 1)


y_4 = y_4.replace(0, 1)
# y_4 = y_4.replace(1, 1)
y_4 = y_4.replace(2, 1)
y_4 = y_4.replace(3, 1)
y_4 = y_4.replace(4, 0)



df = df.assign(Label = y)

#### Split dataset

In [67]:

split = 0.7

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=split)

In [68]:

label_counts2 = Counter(y)
print(label_counts2)


Counter({0: 7804, 1: 5334, 2: 1415, 3: 438, 4: 9})


In [69]:
X_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
12261,-0.110249,-0.007723,-0.004849,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
95229,-0.109865,-0.007536,-0.004836,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
25115,-0.110249,-0.007727,-0.004835,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
105746,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
26007,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138967,-0.155534,-0.019856,-0.096896,-0.017624,-0.059104,-0.019459,-0.113521,-0.143999,-0.890373,-0.016494,...,-0.453815,-0.18843,-0.009419,-0.174880,-0.313124,-0.030535,-0.025803,-0.105681,0.718027,-0.056997
108216,-0.110249,-0.007715,-0.003932,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
73259,-0.110249,-0.007726,-0.003903,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,-0.618438,-0.053906,-0.031767,-0.019726,0.825150,-0.046432
28009,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,-0.312889,-0.11205,-0.028606,-0.139982,1.616978,-0.053906,-0.031767,-0.019726,-1.211901,-0.046432


In [70]:
y_train

12261     0
95229     0
25115     0
105746    1
26007     1
         ..
138967    1
108216    0
73259     0
28009     1
16436     1
Name: Label, Length: 10500, dtype: int64

## LEVEL 0 - Weak models - Base Learner

In [71]:
with open(output_file_name, "a") as f: print('------------START of WEAK LEARNERS (BASE MODELS) - STACK 00 -----------------', file = f)

#Defining Basemodels

rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
lgbm = LGBMClassifier()
knn_clf=KNeighborsClassifier(n_neighbors = 5)

# SVM
clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)


#DNN

#Model Parameters
dropout_rate = 0.2
nodes = 3
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=100
batch_size=128


num_columns = X_train.shape[1]

dnn = tf.keras.Sequential()

# Input layer
dnn.add(tf.keras.Input(shape=(num_columns,)))
# Dense layers with dropout
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))
dnn.add(tf.keras.layers.Dense(nodes))
dnn.add(tf.keras.layers.Dropout(dropout_rate))

# Output layer
dnn.add(tf.keras.layers.Dense(out_layer, activation='softmax'))

dnn.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

dnn.summary()



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 3)                 369       
_________________________________________________________________
dropout_10 (Dropout)         (None, 3)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 12        
_________________________________________________________________
dropout_11 (Dropout)         (None, 3)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 12        
_________________________________________________________________
dropout_12 (Dropout)         (None, 3)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                

In [72]:
#Training Basemodels
n_splits = 5  # You can adjust the number of folds as needed

print('---------------------------------------------------------------------------------')
print('Training Model')
with open(output_file_name, "a") as f: print('Training weak models - level 0', file = f)
print('---------------------------------------------------------------------------------')

if use_model_ada == 1 and load_model_ada == 0:

    print('---------------------------------------------------------------------------------')
    print('Training ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA
    start = time.time()
    ada = abc.fit(X_train, y_train)
    end = time.time()

    ada_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)

    # Assuming 'model' is your trained model
    joblib.dump(ada, 'ada_base_model.joblib')


if use_model_rf == 1 and load_model_rf == 0:

    print('---------------------------------------------------------------------------------')
    print('Training RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('Training RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    model_rf = rf.fit(X_train,y_train)
    end = time.time()

    rf_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(model_rf, 'rf_base_model.joblib')

if use_model_svm == 1 and load_model_svm == 0:

    print('---------------------------------------------------------------------------------')
    print('Training SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM

    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    # clf.score(X_train, y_train)
    svm_tr_time_taken= time_taken = end - start

    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(clf, 'svm_base_model.joblib')


if use_model_knn == 1 and load_model_knn == 0:

    #KNN
    print('---------------------------------------------------------------------------------')
    print('Training KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training KNN', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    knn_clf.fit(X_train,y_train)
    end = time.time()

    knn_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(knn_clf, 'knn_base_model.joblib')


if use_model_lgbm == 1 and load_model_lgbm == 0:

    print('---------------------------------------------------------------------------------')
    print('Training LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    start = time.time()
    lgbm.fit(X_train, y_train)
    end = time.time()

    lgbm_tr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(lgbm, 'lgbm_base_model.joblib')

if use_model_mlp == 1 and load_model_mlp == 0:


    print('---------------------------------------------------------------------------------')
    print('Training MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training MLP', file = f)
    print('---------------------------------------------------------------------------------')

    start = time.time()
    MLP = mlp.fit(X_train, y_train)
    end = time.time()

    mlp_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)
    joblib.dump(MLP, 'mlp_base_model.joblib')


if use_model_dnn == 1 and load_model_dnn == 0:
    from keras.callbacks import EarlyStopping

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    print('---------------------------------------------------------------------------------')
    print('Training DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training DNN', file = f)
    print('---------------------------------------------------------------------------------')

    # Start the timer
    start = time.time()
    dnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])

    # End the timer
    end = time.time()

    dnn_tr_time_taken= time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)


with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)



---------------------------------------------------------------------------------
Training Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training ADA
---------------------------------------------------------------------------------


---------------------------------------------------------------------------------
Training RF
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training SVM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training KNN
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training LGBM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training MLP
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training DNN
--------------------

In [73]:
# Not implemented, the idea was to save and load the model if you did not want to run it again. 

if load_model_ada == 1:
    ada = joblib.load('ada_base_model.joblib')

if load_model_svm == 1:
    clf =  joblib.load('svm_base_model.joblib')

if load_model_dnn == 1:
    dnn = load_model("DNN_base_model.h5")

if load_model_knn == 1:
    knn_clf = joblib.load('knn_base_model.joblib')

if load_model_mlp == 1:
    MLP = joblib.load('mlp_base_model.joblib')

if load_model_rf == 1:
    rf = joblib.load('rf_base_model.joblib')

if load_model_lgbm == 1:
    lgbm = joblib.load('lgbm_base_model.joblib')







### Base leaners predictions

In [74]:

with open(output_file_name, "a") as f: print('Generating Predictions', file = f)

if use_model_rf == 1:

    print('---------------------------------------------------------------------------------')
    print('Prediction RF')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction RF', file = f)
    print('---------------------------------------------------------------------------------')
    #RF
    start = time.time()
    preds_rf = rf.predict(X_test)
    preds_rf_prob = rf.predict_proba(X_test)
    end = time.time()
    rf_pr_time_taken=  time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_svm == 1:

    print('Prediction SVM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction SVM', file = f)
    print('---------------------------------------------------------------------------------')
    #SVM
    start = time.time()
    preds_svm = clf.predict(X_test)
    # preds_svm_prob = clf.predict_proba(X_test)

    #Since SVM does not deal with prob by nature we use a meta learner
    # https://stackoverflow.com/questions/55250963/how-to-get-probabilities-for-sgdclassifier-linearsvm

    model = CalibratedClassifierCV(clf)

    model.fit(X, y)
    preds_svm_prob = model.predict_proba(X)

    end = time.time()
    svm_pr_time_taken = time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_lgbm == 1:

    print('Prediction LGBM')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction LGBM', file = f)
    print('---------------------------------------------------------------------------------')
    #LGBM
    start = time.time()
    preds_lgbm = lgbm.predict(X_test)
    preds_lgbm_prob = lgbm.predict_proba(X_test)

    end = time.time()
    lgbm_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_dnn == 1:

    print('Prediction DNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction DNN', file = f)
    print('---------------------------------------------------------------------------------')
    #DNN
    start = time.time()
    pred_dnn = dnn.predict(X_test)
    preds_dnn_prob = pred_dnn
    preds_dnn = np.argmax(pred_dnn,axis = 1)
    end = time.time()
    dnn_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')

if use_model_ada == 1:

    print('Prediction ADA')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction ADA', file = f)
    print('---------------------------------------------------------------------------------')
    #ADA
    start = time.time()
    preds_ada = ada.predict(X_test)
    preds_ada_prob = ada.predict_proba(X_test)

    end = time.time()
    ada_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction MLP')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction MLP', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_mlp == 1:

    #MLP
    start = time.time()
    y_pred = MLP.predict_proba(X_test)
    preds_mlp_prob = y_pred
    preds_mlp = np.argmax(y_pred,axis = 1)
    end = time.time()
    mlp_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    print('---------------------------------------------------------------------------------')
    print('Prediction KNN')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Prediction KNN', file = f)
    print('---------------------------------------------------------------------------------')

if use_model_knn == 1:

    #KNN
    start = time.time()
    preds_knn =knn_clf.predict(X_test)
    preds_knn_prob =knn_clf.predict_proba(X_test)

    preds_knn
    end = time.time()
    knn_pr_time_taken=time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)


---------------------------------------------------------------------------------
Prediction RF
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction SVM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction LGBM
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction DNN
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction ADA
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Prediction MLP
--------

In [75]:
preds_3 = np.argmax(preds_svm_prob,axis = 1)


### METRICS - Base Learners

In [76]:
#RF
if use_model_rf == 1:
    # start = time.time()
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('RF base model', file = f)

    pred_label = preds_rf
    name = 'rf'
    metrics = confusion_metrics(name, pred_label, y_test,rf_pr_time_taken + rf_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    globals()[f"{name}_time_00"] = rf_pr_time_taken + rf_tr_time_taken


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2    3    4
0  2363.0     1.0    3.0  0.0  0.0
1    43.0  1551.0    9.0  0.0  0.0
2    33.0    10.0  363.0  0.0  0.0
3   109.0     0.0    6.0  8.0  0.0
4     1.0     0.0    0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9522222222222222
Precision total:  0.7745487719830721
Recall total:  0.5850000481965352
F1 total:  0.597214670613797
BACC total:  0.5850000481965352
MCC total:  0.9186776002432332


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [77]:
#DNN
if use_model_dnn == 1:
    start = time.time()
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('DNN base model', file = f)


    pred_label = preds_dnn
    name = 'dnn'
    metrics = confusion_metrics(name, pred_label, y_test, dnn_pr_time_taken + dnn_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = dnn_pr_time_taken + dnn_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1    2    3    4
0  2331.0    36.0  0.0  0.0  0.0
1    62.0  1541.0  0.0  0.0  0.0
2   282.0   124.0  0.0  0.0  0.0
3   118.0     5.0  0.0  0.0  0.0
4     1.0     0.0  0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8604444444444445
Precision total:  0.3475140583447532
Recall total:  0.38922267895984003
F1 total:  0.3669425051275786
BACC total:  0.38922267895984003
MCC total:  0.7582984984810872


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [78]:
#ADA
if use_model_ada == 1:
    start = time.time()
    
    pred_label = preds_ada
    name = 'ada'
    metrics = confusion_metrics(name, pred_label, y_test, ada_pr_time_taken + ada_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = ada_pr_time_taken + ada_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0      1       2    3    4
0  2122.0  162.0    78.0  5.0  0.0
1   150.0   65.0  1384.0  4.0  0.0
2   155.0    6.0   242.0  3.0  0.0
3   109.0    7.0     7.0  0.0  0.0
4     1.0    0.0     0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.5397777777777778
Precision total:  0.24973841173618946
Recall total:  0.30662030712139987
F1 total:  0.2329157135863175
BACC total:  0.30662030712139987
MCC total:  0.33871380906463655


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [79]:
#SVM
if use_model_svm == 1:
    start = time.time()

    pred_label = preds_svm
    name = 'svm'
    metrics = confusion_metrics(name, pred_label, y_test,svm_pr_time_taken + svm_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = svm_pr_time_taken + svm_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2     3    4
0  2290.0    28.0   23.0  26.0  0.0
1    25.0  1577.0    1.0   0.0  0.0
2    29.0    16.0  359.0   2.0  0.0
3    33.0     2.0    0.0  88.0  0.0
4     0.0     0.0    0.0   0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9588888888888889
Precision total:  0.9262028343084049
Recall total:  0.9101866779825443
F1 total:  0.9179051407788913
BACC total:  0.9101866779825443
MCC total:  0.9298092147834897


In [80]:
#KNN
if use_model_knn == 1:
    start = time.time()
    pred_label = preds_knn
    name = 'knn'
    metrics = confusion_metrics(name, pred_label, y_test,knn_pr_time_taken + knn_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = knn_pr_time_taken + knn_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2332.0    15.0   11.0    9.0  0.0
1     2.0  1598.0    2.0    1.0  0.0
2    12.0    17.0  377.0    0.0  0.0
3    12.0     0.0    0.0  111.0  0.0
4     1.0     0.0    0.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9817777777777777
Precision total:  0.7705889217922
Recall total:  0.7626209303206533
F1 total:  0.7665017702069287
BACC total:  0.7626209303206533
MCC total:  0.9689838564473047


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [81]:
#MLP
if use_model_mlp == 1:
    start = time.time()
    pred_label = preds_mlp
    name = 'mlp'
    metrics = confusion_metrics(name, pred_label, y_test,mlp_pr_time_taken + mlp_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start
    globals()[f"{name}_time_00"] = mlp_pr_time_taken + mlp_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2339.0     8.0    5.0   15.0  0.0
1     4.0  1599.0    0.0    0.0  0.0
2    12.0     5.0  386.0    3.0  0.0
3     9.0     0.0    0.0  113.0  1.0
4     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9862222222222222
Precision total:  0.8662335767649678
Recall total:  0.9710226924322607
F1 total:  0.9017144796373003
BACC total:  0.9710226924322607
MCC total:  0.9765614002651856


In [82]:
#lgbm
start_lgbm = time.time()
if use_model_lgbm == 1:

    pred_label = preds_lgbm
    name = 'lgbm'
    metrics = confusion_metrics(name, pred_label, y_test,lgbm_pr_time_taken + lgbm_tr_time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    end = time.time()
    time_taken = end - start_lgbm
    globals()[f"{name}_time_00"] = lgbm_pr_time_taken + lgbm_tr_time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2363.0     2.0    1.0    1.0  0.0
1     6.0  1596.0    0.0    0.0  1.0
2     6.0     2.0  398.0    0.0  0.0
3     7.0     0.0    0.0  116.0  0.0
4     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9942222222222222
Precision total:  0.8956940470889165
Recall total:  0.9834656564678221
F1 total:  0.9227751607758353
BACC total:  0.9834656564678221
MCC total:  0.9901596923430213


### Decision Tree

In [83]:

start = time.time()

# Create a Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
# Train the classifier on the training data
dt_classifier.fit(X_train, y_train)
# Make predictions on the test data
preds_dt = dt_classifier.predict(X_test)
# Evaluate the accuracy of the model
preds_dt_prob = dt_classifier.predict_proba(X_test)


pred_label = preds_dt
name = 'dt'


end = time.time()
time_taken = end - start

metrics = confusion_metrics(name, pred_label, y_test,time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    

globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2342.0     7.0   11.0    6.0  1.0
1     8.0  1592.0    3.0    0.0  0.0
2    17.0     3.0  386.0    0.0  0.0
3     5.0     2.0    1.0  115.0  0.0
4     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9857777777777778
Precision total:  0.8785755775571594
Recall total:  0.973654847931759
F1 total:  0.9094282947512642
BACC total:  0.973654847931759
MCC total:  0.9757737122479826


### CATBOOST

In [84]:

start = time.time()
cat_00 = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')

# Fit the model
cat_00.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=10)

# Make predictions on the test set
preds_cat = cat_00.predict(X_test)
preds_cat_prob = cat_00.predict_proba(X_test)
preds_cat = np.squeeze(preds_cat)


if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Catboost base model', file = f)
    pred_label = preds_cat
    # pred_label = y_pred
    end = time.time()
    time_taken = end - start
    name = 'cat'
    metrics = confusion_metrics(name, pred_label, y_test, time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC

    globals()[f"{name}_time_00"] = time_taken


0:	learn: 1.3205825	test: 1.3188172	best: 1.3188172 (0)	total: 16.3ms	remaining: 1.62s
10:	learn: 0.4449491	test: 0.4413229	best: 0.4413229 (10)	total: 96.7ms	remaining: 782ms
20:	learn: 0.2321835	test: 0.2317243	best: 0.2317243 (20)	total: 154ms	remaining: 580ms
30:	learn: 0.1484856	test: 0.1497024	best: 0.1497024 (30)	total: 209ms	remaining: 466ms
40:	learn: 0.1087867	test: 0.1119304	best: 0.1119304 (40)	total: 263ms	remaining: 379ms
50:	learn: 0.0885619	test: 0.0927513	best: 0.0927513 (50)	total: 316ms	remaining: 304ms
60:	learn: 0.0755467	test: 0.0795962	best: 0.0795962 (60)	total: 369ms	remaining: 236ms
70:	learn: 0.0652197	test: 0.0697321	best: 0.0697321 (70)	total: 422ms	remaining: 172ms
80:	learn: 0.0595860	test: 0.0644432	best: 0.0644432 (80)	total: 474ms	remaining: 111ms
90:	learn: 0.0553427	test: 0.0607782	best: 0.0607782 (90)	total: 525ms	remaining: 52ms
99:	learn: 0.0512405	test: 0.0568618	best: 0.0568618 (99)	total: 572ms	remaining: 0us

bestTest = 0.05686181247
bestItera

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [85]:

start = time.time()

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # for multi-class classification
    'num_class': 5,  # specify the number of classes
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'  # metric for multi-class classification
}

# Train the XGBoost model
num_round = 100
xgb_00 = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
preds_xgb = xgb_00.predict(dtest)


# Get class probabilities
# Assuming binary classification, get the probability for the positive class (class 1)
preds_xgb_margin = xgb_00.predict(dtest, output_margin=True)
preds_xgb_prob = 1 / (1 + np.exp(-preds_xgb_margin))




In [86]:

if 1 == 1:

    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('xgboost base model', file = f)




    pred_label = preds_xgb
    name = 'xgb'

    end = time.time()
    time_taken = end - start
    metrics = confusion_metrics(name, pred_label, y_test,time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC

    globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0.0     1.0    2.0    3.0  4.0
0.0  2350.0     6.0    2.0    9.0  0.0
1.0     5.0  1598.0    0.0    0.0  0.0
2.0    10.0     3.0  391.0    2.0  0.0
3.0     5.0     0.0    1.0  117.0  0.0
4.0     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9904444444444445
Precision total:  0.9784817940829201
Recall total:  0.9807944921532954
F1 total:  0.9795197389737463
BACC total:  0.9807944921532954
MCC total:  0.9837320701487882


### LR

In [87]:


#Logistic Regression
print('---------------------------------------------------------------------------------')
print('Defining Logistic Regression Model')
print('---------------------------------------------------------------------------------')
logreg_00 = LogisticRegression()

if 1 == 1 and 0 == 0:
    print('---------------------------------------------------------------------------------')
    print('Training LR ')
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print('Training LR', file = f)
    print('---------------------------------------------------------------------------------')
    start_lr = start = time.time()
    logreg_00.fit(X_train,y_train)
    end = time.time()

    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed training time ', time_taken, file = f)


if 1 == 1:

    #lR
    start = time.time()
    preds_lr = preds_logreg =logreg_00.predict(X_test)
    preds_lr_prob = logreg_00.predict_proba(X_test)
    end = time.time()
    time_taken = end - start
    with open(output_file_name, "a") as f: print('Elapsed prediction time ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('---------------------------------------------------------------------------------', file = f)

#LR
if 1 == 1:
    pred_label = preds_logreg
    name = 'lr'

    end = time.time()
    time_taken = end - start_lr
    metrics = confusion_metrics(name, pred_label, y_test, time_taken)

    Acc = metrics[0]
    Precision = metrics[1]
    Recall = metrics[2]
    F1 = metrics[3]
    BACC = metrics[4]
    MCC = metrics[5]    

    globals()[f"{name}_acc_00"] = Acc
    globals()[f"{name}_pre_00"] = Precision
    globals()[f"{name}_rec_00"] = Recall
    globals()[f"{name}_f1_00"] = F1
    globals()[f"{name}_bacc_00"] = BACC
    globals()[f"{name}_mcc_00"] = MCC
    

    globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
Defining Logistic Regression Model
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Training LR 
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2312.0    26.0   14.0   15.0  0.0
1    13.0  1589.0    1.0    0.0  0.0
2    21.0     7.0  375.0    3.0  0.0
3    20.0     2.0    1.0  100.0  0.0
4     1.0     0.0    0.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9724444444444444
Precision tot

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging DT  


In [88]:

start = time.time()
# # Define the base classifier (Decision Tree in this case)
base_classifier = DecisionTreeClassifier(random_state=42)

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)


pred_label = y_pred
name = 'bag_dt'
end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    

globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2359.0     3.0    2.0    3.0  0.0
1     6.0  1597.0    0.0    0.0  0.0
2    14.0     3.0  389.0    0.0  0.0
3     6.0     0.0    1.0  116.0  0.0
4     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9915555555555555
Precision total:  0.990498481065547
Recall total:  0.9788189444283896
F1 total:  0.9845434330490013
BACC total:  0.9788189444283896
MCC total:  0.9856103618364205


## bagging SVM

In [89]:
start = time.time()


# Instantiate the SGDClassifier with additional hyperparameters
svm_01 = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)

# # Define the base classifier (Decision Tree in this case)
base_classifier = svm_01

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)


with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_svm'
pred_label = y_pred
end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test,time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC


globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2     3    4
0  2294.0    26.0   26.0  21.0  0.0
1    24.0  1576.0    3.0   0.0  0.0
2    33.0    14.0  359.0   0.0  0.0
3    44.0     1.0    0.0  78.0  0.0
4     0.0     0.0    0.0   0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9573333333333334
Precision total:  0.9291219466148519
Recall total:  0.8941397298834047
F1 total:  0.9098655073869489
BACC total:  0.8941397298834047
MCC total:  0.9269937310596706


## Bagging MLP

In [90]:

start = time.time()
# create MLPClassifier instance
mlp_00 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)

base_classifier = mlp_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)


with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_mlp'
pred_label = y_pred
end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken


Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2339.0     9.0    5.0   14.0  0.0
1     4.0  1599.0    0.0    0.0  0.0
2    13.0     3.0  388.0    2.0  0.0
3     8.0     0.0    0.0  114.0  1.0
4     0.0     0.0    0.0    0.0  1.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9868888888888889
Precision total:  0.8692352689865317
Recall total:  0.9736339303673001
F1 total:  0.9045771058188462
BACC total:  0.9736339303673001
MCC total:  0.9776943846035531


### bagging KNN

In [91]:

knn_00=KNeighborsClassifier(n_neighbors = 5)
start = time.time()
base_classifier = knn_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_00, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_knn'

pred_label = y_pred
end = time.time()
time_taken = end - start

metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2326.0    18.0   12.0   11.0  0.0
1     1.0  1599.0    2.0    1.0  0.0
2    11.0    18.0  377.0    0.0  0.0
3    13.0     0.0    0.0  110.0  0.0
4     1.0     0.0    0.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9804444444444445
Precision total:  0.7665521894542537
Recall total:  0.7606127092749453
F1 total:  0.7634911269342327
BACC total:  0.7606127092749453
MCC total:  0.9667499221642365


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



### bag LogRegression


In [92]:

start = time.time()
#Logistic Regression
print('---------------------------------------------------------------------------------')
print('Defining baggin Logistic Regression Model')
print('---------------------------------------------------------------------------------')
logreg_00 = LogisticRegression()


base_classifier = logreg_00

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Evaluate accuracy
# accuracy = accuracy_score(y_test_00, y_pred)
# print(f'Accuracy: {accuracy}')

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_lr'

pred_label = y_pred

end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken

---------------------------------------------------------------------------------
Defining baggin Logistic Regression Model
---------------------------------------------------------------------------------


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2      3    4
0  2311.0    27.0   14.0   15.0  0.0
1    16.0  1586.0    1.0    0.0  0.0
2    21.0     7.0  374.0    4.0  0.0
3    19.0     2.0    1.0  101.0  0.0
4     1.0     0.0    0.0    0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9715555555555555
Precision total:  0.750875051697339
Recall total:  0.7416113444710273
F1 total:  0.7461350535549103
BACC total:  0.7416113444710273
MCC total:  0.9515090692138751


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging RF

In [93]:
start = time.time()


rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)

base_classifier = rf

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_rf'

pred_label = y_pred

end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2    3    4
0  2363.0     2.0    2.0  0.0  0.0
1    70.0  1533.0    0.0  0.0  0.0
2    42.0     8.0  356.0  0.0  0.0
3   108.0     3.0    3.0  9.0  0.0
4     1.0     0.0    0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.9468888888888889
Precision total:  0.7784428943605695
Recall total:  0.5809319994492763
F1 total:  0.5985705489787949
BACC total:  0.5809319994492763
MCC total:  0.9097403802504108


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging ADA

In [94]:
start = time.time()


ada = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)

base_classifier = ada

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_ada'

pred_label = y_pred

end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0      1      2     3    4
0  2060.0  186.0  116.0   4.0  1.0
1    54.0  781.0  768.0   0.0  0.0
2   123.0   12.0  271.0   0.0  0.0
3    60.0   17.0   10.0  36.0  0.0
4     1.0    0.0    0.0   0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.6995555555555556
Precision total:  0.5626372503314618
Recall total:  0.4635364095577231
F1 total:  0.45417878409795076
BACC total:  0.4635364095577231
MCC total:  0.5429096194742403


### Bagging LGBM

In [95]:
start = time.time()


lgbm = LGBMClassifier()


base_classifier = lgbm

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_lgbm'

pred_label = y_pred

end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken



---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
        0       1      2     3    4
0  2358.0     7.0    0.0   2.0  0.0
1    17.0  1586.0    0.0   0.0  0.0
2    22.0     4.0  380.0   0.0  0.0
3    35.0     2.0    0.0  86.0  0.0
4     1.0     0.0    0.0   0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.98
Precision total:  0.7876633010809426
Recall total:  0.7241480372450987
F1 total:  0.7510435704170609
BACC total:  0.7241480372450987
MCC total:  0.9658935534784219


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging Catboost 

In [96]:

start = time.time()

bag_cat = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')

base_classifier = bag_cat

# Define the BaggingClassifier
bagging_classifier = BaggingClassifier(base_classifier, n_estimators=10, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

name = 'bag_cat'

pred_label = y_pred

end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test, time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken



0:	learn: 1.3254632	total: 17ms	remaining: 1.68s
1:	learn: 1.1239883	total: 30.5ms	remaining: 1.5s
2:	learn: 0.9952761	total: 42.5ms	remaining: 1.37s
3:	learn: 0.8791508	total: 51.3ms	remaining: 1.23s
4:	learn: 0.7899724	total: 59.8ms	remaining: 1.14s
5:	learn: 0.7086156	total: 66.7ms	remaining: 1.04s
6:	learn: 0.6408340	total: 73.8ms	remaining: 980ms
7:	learn: 0.5842988	total: 80ms	remaining: 920ms
8:	learn: 0.5328991	total: 86.2ms	remaining: 872ms
9:	learn: 0.4885743	total: 92.5ms	remaining: 832ms
10:	learn: 0.4540532	total: 98.1ms	remaining: 794ms
11:	learn: 0.4219235	total: 103ms	remaining: 758ms
12:	learn: 0.3913296	total: 109ms	remaining: 732ms
13:	learn: 0.3638104	total: 115ms	remaining: 706ms
14:	learn: 0.3415363	total: 121ms	remaining: 683ms
15:	learn: 0.3193731	total: 127ms	remaining: 665ms
16:	learn: 0.2991671	total: 132ms	remaining: 644ms
17:	learn: 0.2816184	total: 137ms	remaining: 625ms
18:	learn: 0.2651195	total: 142ms	remaining: 606ms
19:	learn: 0.2513871	total: 148ms	r

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


### Bagging Combined

In [97]:
### Bagging with many models
##### do bootstrapping 
##### 1. Multiple subsets are created from the original dataset, selecting observations with replacement.

start = time.time()

num_bootstraps = 10  # Adjust the number of bootstraps as needed

original_data_df = X_train.assign(label = y_train)
boot_df = []
for i in range(0,num_bootstraps): 
    boot_df.append(original_data_df.sample(frac = 1, replace=True).reset_index(drop=True))

# boot_df[5]

#### 2.A base model (weak model) is created on each of these subsets.
bag_comb_pred = []

# SVM

clf = SGDClassifier(
    loss='hinge',           # hinge loss for linear SVM
    penalty='l2',           # L2 regularization to prevent overfitting
    alpha=1e-4,             # Learning rate (small value for fine-grained updates)
    max_iter=1000,          # Number of passes over the training data
    random_state=42,        # Seed for reproducible results
    learning_rate='optimal' # Automatically adjusts the learning rate based on the training data
)
y_train_boot = boot_df[0].pop('label')
X_train_boot = boot_df[0]
clf.fit(X_train_boot, y_train_boot)
preds_svm_00 = clf.predict(X_test)
bag_comb_pred.append(preds_svm_00)




#ADA

abc = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
ada = abc.fit(X_train, y_train)
y_train_boot = boot_df[1].pop('label')
X_train_boot = boot_df[1]
preds_ada_00 = ada.predict(X_test)
bag_comb_pred.append(preds_ada_00)

#Catboost

cat_00 = catboost.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', custom_metric='Accuracy')
y_train_boot = boot_df[2].pop('label')
X_train_boot = boot_df[2]
cat_00.fit(X_train_boot, y_train_boot, eval_set=(X_test, y_test), verbose=10)
preds_cat = cat_00.predict(X_test)
preds_cat = np.squeeze(preds_cat)
pred_label = preds_cat
bag_comb_pred.append(preds_cat)

#MLP

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=1)
y_train_boot = boot_df[3].pop('label')
X_train_boot = boot_df[3]
if 1 == 1 and 0 == 0:
    MLP = mlp.fit(X_train_boot, y_train_boot)
    y_pred = MLP.predict_proba(X_test)
    preds_mlp_00 = np.argmax(y_pred,axis = 1)

bag_comb_pred.append(preds_mlp_00)

#LGBM

lgbm = LGBMClassifier()
y_train_boot = boot_df[4].pop('label')
X_train_boot = boot_df[4]

if 1 == 1 and 0 == 0:
    lgbm.fit(X_train_boot, y_train_boot)
    preds_lgbm_00 = lgbm.predict(X_test)
    bag_comb_pred.append(preds_lgbm_00)
#KNN

knn_clf_00=KNeighborsClassifier(n_neighbors = 5)
y_train_boot = boot_df[5].pop('label')
X_train_boot = boot_df[5]

if 1 == 1 and 0 == 0:
    knn_clf_00.fit(X_train_boot,y_train_boot)
if use_model_knn == 1:
    preds_knn =knn_clf_00.predict(X_test)
    bag_comb_pred.append(preds_knn)
#Random Forest

rf = RandomForestClassifier(max_depth = 5,  n_estimators = 10, min_samples_split = 2, n_jobs = -1)
y_train_boot = boot_df[6].pop('label')
X_train_boot = boot_df[6]

if True == True:
    model_rf_00 = rf.fit(X_train_boot,y_train_boot)
    preds_rf_00 = model_rf_00.predict(X_test)
    bag_comb_pred.append(preds_rf_00)
#DNN

#Model Parameters
y_train_boot = boot_df[7].pop('label')
X_train_boot = boot_df[7]


dropout_rate = 0.2
nodes = 3
out_layer = 5
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=100
batch_size=128
num_columns = X_train_boot.shape[1]
dnn_00 = tf.keras.Sequential()
# Input layer
dnn_00.add(tf.keras.Input(shape=(num_columns,)))
# Dense layers with dropout
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
dnn_00.add(tf.keras.layers.Dense(nodes))
dnn_00.add(tf.keras.layers.Dropout(dropout_rate))
# Output layer
# dnn_00.add(tf.keras.layers.Dense(out_layer))
dnn_00.add(tf.keras.layers.Dense(out_layer, activation='softmax'))
dnn_00.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
dnn_00.fit(X_train_boot, y_train_boot, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])
pred_dnn = dnn_00.predict(X_test)
preds_dnn_00 = np.argmax(pred_dnn,axis = 1)
bag_comb_pred.append(preds_dnn_00)
#LogReg

logreg_00 = LogisticRegression()
y_train_boot = boot_df[8].pop('label')
X_train_boot = boot_df[8]

logreg_00.fit(X_train_boot,y_train_boot)
preds_logreg =logreg_00.predict(X_test)
bag_comb_pred.append(preds_logreg)

y_train_boot = boot_df[9].pop('label')
X_train_boot = boot_df[9]

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_boot, label=y_train_boot)
dtest = xgb.DMatrix(X_test, label=y_test)
# Set XGBoost parameters
params = {
    'objective': 'multi:softmax',  # for multi-class classification
    'num_class': 5,  # specify the number of classes
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'mlogloss'  # metric for multi-class classification
}
# Train the XGBoost model
num_round = 100
xgb_00 = xgb.train(params, dtrain, num_round)
preds_xgb_00 = xgb_00.predict(dtest)
bag_comb_pred.append(preds_xgb_00)
### 3. The models run in parallel and are independent of each other.
bag_vot_df = pd.DataFrame()
for i in range(0,len(bag_comb_pred)):
    bag_vot_df[f'model_{i}'] =  bag_comb_pred[i]
print(bag_vot_df)
# Voting start


# bag_comb_pred_df = pd.DataFrame(bag_comb_pred)
# Extract predictions columns

# predictions = df[['dnn', 'rf', 'lgbm', 'ada', 'knn', 'mlp', 'svm','cat','xgb']]
    # selected_columns = df.loc[:, ~df.columns.isin(['rf'])]
predictions = bag_vot_df 

# predictions = bag_comb_pred_df.loc[:, ~df.columns.isin(['label'])] #df[column_features]

# Use the mode function along axis 1 to get the most common prediction for each row
ensemble_predictions, _ = mode(predictions.values, axis=1)

# Add the ensemble predictions to the DataFrame
bag_vot_df['ensemble'] = ensemble_predictions.astype(int)

# Display the DataFrame with ensemble predictions
print(bag_vot_df)

pred_label = bag_vot_df ['ensemble'].values
bag_vot_df.pop('ensemble')


name='bag_comb'
end = time.time()
time_taken = end - start
metrics = confusion_metrics(name, pred_label, y_test,time_taken)

Acc = metrics[0]
Precision = metrics[1]
Recall = metrics[2]
F1 = metrics[3]
BACC = metrics[4]
MCC = metrics[5]    


globals()[f"{name}_acc_00"] = Acc
globals()[f"{name}_pre_00"] = Precision
globals()[f"{name}_rec_00"] = Recall
globals()[f"{name}_f1_00"] = F1
globals()[f"{name}_bacc_00"] = BACC
globals()[f"{name}_mcc_00"] = MCC

globals()[f"{name}_time_00"] = time_taken



0:	learn: 1.3047254	test: 1.3016224	best: 1.3016224 (0)	total: 19.2ms	remaining: 1.9s
10:	learn: 0.4533872	test: 0.4514314	best: 0.4514314 (10)	total: 123ms	remaining: 995ms
20:	learn: 0.2339763	test: 0.2357504	best: 0.2357504 (20)	total: 181ms	remaining: 682ms
30:	learn: 0.1480733	test: 0.1512366	best: 0.1512366 (30)	total: 234ms	remaining: 520ms
40:	learn: 0.1124202	test: 0.1179599	best: 0.1179599 (40)	total: 282ms	remaining: 406ms
50:	learn: 0.0891353	test: 0.0962047	best: 0.0962047 (50)	total: 332ms	remaining: 319ms
60:	learn: 0.0740737	test: 0.0820106	best: 0.0820106 (60)	total: 380ms	remaining: 243ms
70:	learn: 0.0652873	test: 0.0740704	best: 0.0740704 (70)	total: 428ms	remaining: 175ms
80:	learn: 0.0573126	test: 0.0671240	best: 0.0671240 (80)	total: 478ms	remaining: 112ms
90:	learn: 0.0517011	test: 0.0618311	best: 0.0618311 (90)	total: 526ms	remaining: 52.1ms
99:	learn: 0.0476242	test: 0.0581125	best: 0.0581125 (99)	total: 570ms	remaining: 0us

bestTest = 0.05811252376
bestItera

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


      model_0  model_1  model_2  model_3  model_4  model_5  model_6  model_7  \
0           1        2        1        1        1        1        1        1   
1           0        0        0        0        0        0        0        0   
2           1        2        1        1        1        1        1        1   
3           0        0        0        0        1        0        0        0   
4           1        2        2        1        2        1        1        1   
...       ...      ...      ...      ...      ...      ...      ...      ...   
4495        0        1        0        0        0        0        0        0   
4496        1        2        1        1        1        1        1        1   
4497        1        2        1        1        1        1        1        1   
4498        0        0        0        0        0        0        0        0   
4499        0        0        0        0        0        0        0        0   

      model_8  model_9  
0           1 

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Creating new dataset for level 01

In [98]:
# print(len(preds_dnn_prob), len(y_test))

In [99]:
# print(y_test)

In [100]:
df_from_series = y_test.to_frame()
y_test_reset_index = df_from_series.reset_index()
# y_test2 = y_test.reset_index(inplace=True)
# print(y_test_reset_index)
y_test_reset_index.pop('index')

0       125646
1       139912
2        12909
3        93898
4       135355
         ...  
4495     97407
4496      8154
4497     88737
4498      5823
4499    141488
Name: index, Length: 4500, dtype: int64

In [101]:
# y_test_reset_index.values[0][0]

In [102]:
preds_dnn_2 = []
preds_svm_2 = []
preds_rf_2 = []
preds_mlp_2 = []
preds_ada_2 = []
preds_knn_2 = []
preds_lgbm_2 = []
preds_cat_2 = []
preds_xgb_2 = []

preds_lr_2 = []
preds_dt_2 = []

for i in range(0,len(preds_dnn_prob)):  
    # print(i)
    # print(preds_dnn_prob[i][y_test_reset_index.values[i][0]])
    preds_dnn_2.append(preds_dnn_prob[i][y_test_reset_index.values[i][0]])
    preds_svm_2.append(preds_svm_prob[i][y_test_reset_index.values[i][0]])
    preds_rf_2.append(preds_rf_prob[i][y_test_reset_index.values[i][0]])
    preds_mlp_2.append(preds_mlp_prob[i][y_test_reset_index.values[i][0]])
    preds_ada_2.append(preds_ada_prob[i][y_test_reset_index.values[i][0]])
    preds_knn_2.append(preds_knn_prob[i][y_test_reset_index.values[i][0]])
    preds_lgbm_2.append(preds_lgbm_prob[i][y_test_reset_index.values[i][0]])
    preds_cat_2.append(preds_cat_prob[i][y_test_reset_index.values[i][0]])
    preds_xgb_2.append(preds_xgb_prob[i][y_test_reset_index.values[i][0]])
    preds_lr_2.append(preds_lr_prob[i][y_test_reset_index.values[i][0]])
    preds_dt_2.append(preds_dt_prob[i][y_test_reset_index.values[i][0]])

    

In [103]:
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)
with open(output_file_name, "a") as f: print('------------------------------------------------------------------', file = f)

with open(output_file_name, "a") as f: print('------------START of STRONGER LEARNER - STACK 01 -----------------', file = f)


# Stack the vectors horizontally to create a matrix
column_features = ['dnn','rf','lgbm','ada','knn','mlp','svm','cat','xgb','lr','dt','label']
training_matrix2 = np.column_stack((
                          preds_dnn_2,
                          preds_rf_2,
                          preds_lgbm_2,
                          preds_ada_2,
                          preds_knn_2, 
                          preds_mlp_2,
                          preds_svm_2,
                          preds_cat_2,
                          preds_xgb_2,
                          preds_lr_2,
                          preds_dt_2,
                          y_test
                          ))

training_matrix = np.column_stack((
                          preds_dnn,
                          preds_rf,
                          preds_lgbm,
                          preds_ada,
                          preds_knn, 
                          preds_mlp,
                          preds_svm,
                          preds_cat,
                          preds_xgb,
                          preds_lr,
                          preds_dt,
                        #   preds
                          y_test
                          ))
# Print the resulting matrix
print(training_matrix)
print(training_matrix2)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.8599416  0.94578776 0.99999968 ... 0.99488053 1.         1.        ]
 [0.88022864 0.93744969 0.99999313 ... 0.94043086 1.         0.        ]
 [0.92410403 0.76698096 0.99999798 ... 0.99945116 1.         1.        ]
 ...
 [0.94917583 0.92804966 0.99999972 ... 0.99861004 1.         1.        ]
 [0.93674856 0.98348587 0.99999964 ... 0.9912892  1.         0.        ]
 [0.93769014 0.84863876 0.99999949 ... 0.99555077 1.         0.        ]]


In [104]:
df_level_00_0 = pd.DataFrame(training_matrix, columns=column_features)
df_level_00_1 = pd.DataFrame(training_matrix2, columns=column_features)

### Saving the created datasets

In [105]:

# Assuming df is your DataFrame
if feature_selection_bit == 1:

    df_level_00_1.to_csv('base_models_prob_feature_selection.csv', index=False)
    df_level_00_0.to_csv('base_models_class_feature_selection.csv', index=False)
    
if feature_selection_bit == 0:

    df_level_00_1.to_csv('base_models_prob_all_features.csv', index=False)
    df_level_00_0.to_csv('base_models_class_all_features.csv', index=False)

In [106]:
df_level_00_1


,dnn,rf,lgbm,ada,knn,mlp,svm,cat,xgb,lr,dt,label
0,0.859942,0.945788,1.000000,0.035412,1.0,9.999929e-01,6.606513e-01,0.982959,0.991739,0.994881,1.0,1.0
1,0.880229,0.937450,0.999993,0.302884,1.0,9.997380e-01,1.333307e-01,0.988308,0.990285,0.940431,1.0,0.0
2,0.924104,0.766981,0.999998,0.085953,1.0,9.999975e-01,1.154039e-01,0.973105,0.990019,0.999451,1.0,1.0
3,0.945286,0.944461,1.000000,0.315262,1.0,9.999999e-01,8.953582e-02,0.991605,0.977116,0.995532,1.0,0.0
4,0.079151,0.767128,0.999956,0.782876,0.0,7.856260e-07,3.041613e-11,0.964559,0.982889,0.019402,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,0.913306,0.983486,1.000000,0.380782,1.0,9.995879e-01,7.940908e-01,0.995199,0.994188,0.984660,1.0,0.0
4496,0.989246,0.993062,1.000000,0.076962,1.0,1.000000e+00,8.867643e-01,0.996495,0.997377,0.999990,1.0,1.0
4497,0.949176,0.928050,1.000000,0.031356,1.0,1.000000e+00,1.386037e-03,0.982680,0.991739,0.998610,1.0,1.0
4498,0.936749,0.983486,1.000000,0.380437,1.0,9.997988e-01,9.973323e-01,0.996043,0.993793,0.991289,1.0,0.0


In [107]:
df_level_00_0

,dnn,rf,lgbm,ada,knn,mlp,svm,cat,xgb,lr,dt,label
0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4496,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4497,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Now we create performance tables

In [108]:
data = [["" for _ in range(5)] for _ in range(24)]

names_models = ['ADA',
                'SVM',
                'DNN',
                'MLP',
                'KNN',
                'CAT',
                'XGB',
                'LGBM',
                'RF',
                'LR',
                'DT',
                # 'VOTING',
                'Bag_svm',
                'Bag_knn',
                'Bag_DT',
                'Bag_LR',
                'Bag_mlp',

                'Bag_rf',
                'Bag_ada',
                'Bag_lgbm',
                # 'Bag_xgb',
                'Bag_cat',
                'Bag_comb',

                # 'avg',
                # 'weighed_avg'
                ]


level_00_acc = [ada_acc_00,
                svm_acc_00,
                dnn_acc_00,
                mlp_acc_00,
                knn_acc_00,
                cat_acc_00,
                xgb_acc_00,
                lgbm_acc_00,
                rf_acc_00,
                lr_acc_00,
                dt_acc_00,
                # voting_acc_00,
                bag_svm_acc_00,
                bag_knn_acc_00,
                bag_dt_acc_00,
                bag_lr_acc_00,
                bag_mlp_acc_00,
               
                bag_rf_acc_00,
                bag_ada_acc_00,
                bag_lgbm_acc_00,

                bag_cat_acc_00,
                bag_comb_acc_00,
               
                # avg_acc_00,
                # weighed_avg_acc_00
                ]  

                # ]  

level_00_pre = [ada_pre_00,
                svm_pre_00,
                dnn_pre_00,
                mlp_pre_00,
                knn_pre_00,
                cat_pre_00,
                xgb_pre_00,
                lgbm_pre_00,
                rf_pre_00,
                lr_pre_00,
                dt_pre_00,
                # voting_pre_00,
                bag_svm_pre_00,
                bag_knn_pre_00,
                bag_dt_pre_00,
                bag_lr_pre_00,
                bag_mlp_pre_00,

                bag_rf_pre_00,
                bag_ada_pre_00,
                bag_lgbm_pre_00,

                bag_cat_pre_00,
                bag_comb_pre_00,
               
                # avg_pre_00,
                # weighed_avg_pre_00
                ]  

level_00_rec = [ada_rec_00,
                svm_rec_00,
                dnn_rec_00,
                mlp_rec_00,
                knn_rec_00,
                cat_rec_00,
                xgb_rec_00,
                lgbm_rec_00,
                rf_rec_00,
                lr_rec_00,
                dt_rec_00,
                # voting_rec_00,
                bag_svm_rec_00,
                bag_knn_rec_00,
                bag_dt_rec_00,
                bag_lr_rec_00,
                bag_mlp_rec_00,

                bag_rf_rec_00,
                bag_ada_rec_00,
                bag_lgbm_rec_00,

                bag_cat_rec_00,
                bag_comb_rec_00,
               
                # avg_rec_00,
                # weighed_avg_rec_00
                ]  

level_00_f1 = [ada_f1_00,
                svm_f1_00,
                dnn_f1_00,
                mlp_f1_00,
                knn_f1_00,
                cat_f1_00,
                xgb_f1_00,
                lgbm_f1_00,
                rf_f1_00,
                lr_f1_00,
                dt_rec_00,
                # voting_f1_00,
                bag_svm_f1_00,
                bag_knn_f1_00,
                bag_dt_f1_00,
                bag_lr_f1_00,
                bag_mlp_f1_00,

                bag_rf_f1_00,
                bag_ada_f1_00,
                bag_lgbm_f1_00,

                bag_cat_f1_00,
                bag_comb_f1_00,
               
                # avg_f1_00,
                # weighed_avg_f1_00
                ]                   

for i in range(0,len(names_models)):
    data[i][0] =  names_models[i]

    data[i][1] = level_00_acc[i]
    # data[i][2] = level_01_acc[i]

    data[i][2] = level_00_pre[i] 
    # data[i][4] = level_01_pre[i]

    data[i][3] = level_00_rec[i] 
    # data[i][6] = level_01_rec[i]

    data[i][4] = level_00_f1[i]
    # data[i][8] = level_01_f1[i]


headers = ["Models", "ACC-00","PRE-00","REC-00","F1-00"]


# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
print(table)


+----------+--------------------+---------------------+---------------------+---------------------+
| Models   | ACC-00             | PRE-00              | REC-00              | F1-00               |
+==========+====================+=====================+=====================+=====================+
| ADA      | 0.5397777777777778 | 0.24973841173618946 | 0.30662030712139987 | 0.2329157135863175  |
+----------+--------------------+---------------------+---------------------+---------------------+
| SVM      | 0.9588888888888889 | 0.9262028343084049  | 0.9101866779825443  | 0.9179051407788913  |
+----------+--------------------+---------------------+---------------------+---------------------+
| DNN      | 0.8604444444444445 | 0.3475140583447532  | 0.38922267895984003 | 0.3669425051275786  |
+----------+--------------------+---------------------+---------------------+---------------------+
| MLP      | 0.9862222222222222 | 0.8662335767649678  | 0.9710226924322607  | 0.9017144796373003  |


In [109]:
# Combine data into a list of tuples for sorting
model_data = list(zip(names_models, level_00_acc, level_00_pre, level_00_rec, level_00_f1))

# Sort by F1-00 score in descending order
model_data_sorted = sorted(model_data, key=lambda x: x[4], reverse=True)

# Separate the sorted data back into individual lists
sorted_names_models, sorted_level_00_acc, sorted_level_00_pre, sorted_level_00_rec, sorted_level_00_f1 = zip(*model_data_sorted)

# Assign the sorted data to the table
for i in range(len(sorted_names_models)):
    data[i][0] = sorted_names_models[i]
    data[i][1] = sorted_level_00_acc[i]
    data[i][2] = sorted_level_00_pre[i] 
    data[i][3] = sorted_level_00_rec[i] 
    data[i][4] = sorted_level_00_f1[i]

# Define column headers
headers = ["Models", "ACC-00", "PRE-00", "REC-00", "F1-00"]

# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
with open(output_file_name, "a") as f: print('Summary table - LEVEL 00', file = f)

if feature_selection_bit == 1: 
    with open(output_file_name, "a") as f: print('Feature Selection was applied', file = f)
else:
    with open(output_file_name, "a") as f: print('All features were used', file = f)


    
print(table)
with open(output_file_name, "a") as f: print(table, file = f)

+----------+--------------------+---------------------+---------------------+---------------------+
| Models   | ACC-00             | PRE-00              | REC-00              | F1-00               |
+==========+====================+=====================+=====================+=====================+
| Bag_DT   | 0.9915555555555555 | 0.990498481065547   | 0.9788189444283896  | 0.9845434330490013  |
+----------+--------------------+---------------------+---------------------+---------------------+
| XGB      | 0.9904444444444445 | 0.9784817940829201  | 0.9807944921532954  | 0.9795197389737463  |
+----------+--------------------+---------------------+---------------------+---------------------+
| DT       | 0.9857777777777778 | 0.8785755775571594  | 0.973654847931759   | 0.973654847931759   |
+----------+--------------------+---------------------+---------------------+---------------------+
| LGBM     | 0.9942222222222222 | 0.8956940470889165  | 0.9834656564678221  | 0.9227751607758353  |


In [110]:
# implement time table

names_models = ['ADA',
                'SVM',
                'DNN',
                'MLP',
                'KNN',
                'CAT',
                'XGB',
                'LGBM',
                'RF',
                'LR',
                'DT',
                # 'VOTING',
                'Bag_svm',
                'Bag_knn',
                'Bag_DT',
                'Bag_LR',
                'Bag_mlp',

                'Bag_rf',
                'Bag_ada',
                'Bag_lgbm',
                # 'Bag_xgb',
                'Bag_cat',
                'Bag_comb',
                # 'avg',
                # 'weighed_avg'
                ]

data = [["" for _ in range(2)] for _ in range(len(names_models))]

level_00_time = [
                ada_time_00,
                svm_time_00,
                dnn_time_00,
                mlp_time_00,
                knn_time_00,
                cat_time_00,
                xgb_time_00,
                lgbm_time_00,
                rf_time_00,
                lr_time_00,
                dt_time_00,
                # voting_time_00,
                bag_svm_time_00,
                bag_knn_time_00,
                bag_dt_time_00,
                bag_lr_time_00,
                bag_mlp_time_00,

                bag_rf_time_00,
                bag_ada_time_00,
                bag_lgbm_time_00,
                # bag_xgb_time_00,
                bag_cat_time_00,
                bag_comb_time_00,

                # avg_time_00,
                # weighed_avg_time_00
                ]  


# Combine data into a list of tuples for sorting
model_data = list(zip(names_models, level_00_time))

# Sort by F1-00 score in descending order
model_data_sorted = sorted(model_data, key=lambda x: x[1], reverse=False)

# Separate the sorted data back into individual lists
sorted_names_models, sorted_level_00_time = zip(*model_data_sorted)

# Assign the sorted data to the table
for i in range(len(sorted_names_models)):
    data[i][0] = sorted_names_models[i]
    data[i][1] = sorted_level_00_time[i]

# Define column headers
headers = ["Models", "time-00(sec)"]


# Print the table
table = tabulate(data, headers=headers, tablefmt="grid")
with open(output_file_name, "a") as f: print('Time is counted is seconds', file = f)
print(table)
with open(output_file_name, "a") as f: print(table, file = f)
end_program = time.time()
time_program = end_program - start_program
with open(output_file_name, "a") as f: print('Running time of entire program is:', time_program ,' seconds',file = f)

+----------+----------------+
| Models   |   time-00(sec) |
+==========+================+
| DT       |       0.108712 |
+----------+----------------+
| RF       |       0.333082 |
+----------+----------------+
| LR       |       0.454727 |
+----------+----------------+
| Bag_DT   |       0.599282 |
+----------+----------------+
| CAT      |       0.704267 |
+----------+----------------+
| ADA      |       0.746673 |
+----------+----------------+
| LGBM     |       0.917025 |
+----------+----------------+
| XGB      |       1.15176  |
+----------+----------------+
| SVM      |       3.28653  |
+----------+----------------+
| Bag_rf   |       3.66511  |
+----------+----------------+
| Bag_LR   |       4.49088  |
+----------+----------------+
| DNN      |       4.53895  |
+----------+----------------+
| Bag_ada  |       5.36162  |
+----------+----------------+
| Bag_svm  |       5.41128  |
+----------+----------------+
| MLP      |       6.11534  |
+----------+----------------+
| Bag_cat 